# Weight Parameter Study Tutorial

This tutorial demonstrates how to run the Weight Parameter Study experiment, which investigates how the learning rate parameter (w) affects agent learning dynamics, convergence speed, and system performance.

## Concept
The weight parameter controls how quickly agents update their probability distributions based on environmental feedback. Higher weights lead to faster learning but may reduce exploration, whilst lower weights maintain exploration but converge more slowly.

This experiment tests the fundamental exploration-exploitation trade-off in multi-agent learning systems.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime
import sys
import os

# Add the parent directory to the path to import the experiment modules
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))

from experiments.weight_parameter_study import WeightParameterStudy, run_weight_parameter_study
from utils.config import Config
from evaluation.metrics import calculate_entropy, calculate_convergence_speed
from visualisation.plots import plot_parameter_sensitivity, plot_convergence_comparison

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Concept Explanation

### What is the Weight Parameter?
The weight parameter (w) controls the learning intensity in the stochastic learning algorithm. It determines how much agents update their probability distributions based on cost feedback.

**Learning Update Rule:**
p_{i,r}(t+1) = (1-λ_i(t)) * p_{i,r}(t) + λ_i(t) * δ_{r,a_i(t)}")
,where λ_i(t) = w * L_{a_i(t)}(t)

- **Higher w** = Faster learning = Less exploration
- **Lower w** = Slower learning = More exploration

**Study weight parameter effects** \
     - Understanding the exploration-exploitation trade-off \
     - Finding optimal learning rates for different scenarios \
     - Balancing convergence speed with solution quality \
     - Identifying parameter ranges for practical deployment 

**Key research questions** \
     - How does w affect convergence speed? \
     - What is the relationship between w and final system cost? \
     - How does w influence learning stability? \
     - What is the optimal w range for different applications? \
     - How does *w* balance exploration vs exploitation? 
     
**Expected Outcomes** \
     - Higher weights should lead to faster convergence \
     - Lower weights should maintain better exploration \
     - There should be an optimal range balancing speed and quality \
     - Very high weights may lead to instability \
     - Very low weights may be too slow for practical use 


## 2. Parameter Configuration

In this section, we'll configure the experiment parameters. You can modify these values to explore different scenarios.

In [ ]:
# System parameters
num_agents = 10
num_resources = 5
num_iterations = 1000
num_replications = 30

# Weight parameter configuration
weight_values = [0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 0.8, 0.9, 0.95]

# Resource capacity configuration (balanced)
relative_capacity = [1.0/num_resources] * num_resources

# Convergence parameters
convergence_threshold_entropy = 0.1
convergence_threshold_max_prob = 0.9

# Output configuration
output_dir = "results/weight_parameter_tutorial"

# Display configuration
print("=" * 80)
print("EXPERIMENT CONFIGURATION")
print("=" * 80)
print(f"Number of agents: {num_agents}")
print(f"Number of resources: {num_resources}")
print(f"Number of iterations: {num_iterations}")
print(f"Number of replications: {num_replications}")
print(f"Weight values: {weight_values}")
print(f"Relative capacity: {[f'{c:.3f}' for c in relative_capacity]}")
print(f"Convergence entropy threshold: {convergence_threshold_entropy}")
print(f"Convergence max probability threshold: {convergence_threshold_max_prob}")
print(f"Output directory: {output_dir}")

## 3. Running the Experiment

Now we'll create and run the Weight Parameter Study experiment.

In [ ]:
def run_weight_parameter_experiment(params):
    """
    Run the weight parameter study experiment with the given parameters.
    """
    print("=" * 80)
    print("RUNNING WEIGHT PARAMETER STUDY EXPERIMENT")
    print("=" * 80)
    
    # Create output directory
    output_path = Path(params['output_dir'])
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Create custom configuration
    config = Config()
    config.num_agents = params['num_agents']
    config.num_resources = params['num_resources']
    config.relative_capacity = params['relative_capacity']
    config.num_iterations = params['num_iterations']
    config.agent_initialisation_method = "uniform"
    
    # Create and run the experiment
    print(f"Creating WeightParameterStudy with {len(params['weight_values'])} weight values...")
    study = WeightParameterStudy(
        weight_values=params['weight_values'],
        results_dir=params['output_dir'],
        experiment_name="weight_parameter_tutorial"
    )
    
    # Override base config with user parameters
    study.base_config = config
    
    print(f"Running experiment with {params['num_replications']} replications per weight value...")
    print("This may take several minutes depending on the number of replications...")
    
    # Run experiment
    full_results = study.run_experiment(num_episodes=params['num_replications'])
    
    # Convert results to expected format
    study.results = []
    for config_result in full_results['results']:
        config_params = config_result['config_params']
        for episode_result in config_result['episode_results']:
            study.results.append({
                'config_params': config_params,
                'simulation_results': episode_result,
                'replication_id': episode_result['episode']
            })
    
    
    return study

# Create parameter dictionary
params = {
    'num_agents': num_agents,
    'num_resources': num_resources,
    'num_iterations': num_iterations,
    'num_replications': num_replications,
    'weight_values': weight_values,
    'relative_capacity': relative_capacity,
    'convergence_threshold_entropy': convergence_threshold_entropy,
    'convergence_threshold_max_prob': convergence_threshold_max_prob,
    'output_dir': output_dir
}

# Run the experiment
study = run_weight_parameter_experiment(params)

## 4. Generating Analysis and Plots

Now we'll generate comprehensive analysis and create visualisations.

In [ ]:
def generate_analysis_and_plots(study, params):
    """
    Generate comprehensive analysis and create all figures.
    """
    print("=" * 80)
    print("GENERATING ANALYSIS AND PLOTS")
    print("=" * 80)
    
    # Generate detailed analysis
    print("Generating detailed statistical analysis...")
    analysis_results = study.generate_detailed_analysis()
    
    # Create comprehensive plots
    print("Creating comprehensive visualisations...")
    plots_dir = f"{params['output_dir']}/plots"
    plot_files = study.create_comprehensive_plots(plots_dir)
    
    print(f"Generated {len(plot_files)} plot files:")
    for plot_file in plot_files:
        print(f"  - {plot_file}")
    
    # Save results
    print("Saving results and analysis...")
    study.save_results(params['output_dir'])
    
    # Generate comprehensive report
    print("Generating comprehensive analysis report...")
    report = study.generate_comprehensive_report()
    
    # Save report
    report_path = f"{params['output_dir']}/comprehensive_analysis_report.txt"
    with open(report_path, 'w') as f:
        f.write(report)
    
    print(f"Analysis report saved to: {report_path}")
    
    # Display key findings
    print("\n" + "=" * 80)
    print("KEY FINDINGS SUMMARY")
    print("=" * 80)
    
    if 'recommendations' in analysis_results:
        recs = analysis_results['recommendations']
        print(f"Optimal weight for cost performance: {recs.get('optimal_for_cost', 'N/A')}")
        print(f"Optimal weight for convergence speed: {recs.get('optimal_for_convergence', 'N/A')}")
        print(f"Recommended weight range: {recs.get('suggested_range', 'N/A')}")
        print(f"Cost sensitivity: {recs.get('cost_sensitivity', 'N/A')}")
    
    if 'hypothesis_evaluation' in analysis_results:
        print("\nHypothesis Evaluation:")
        hyp_eval = analysis_results['hypothesis_evaluation']
        for key, value in hyp_eval.items():
            status = "✓" if value else "✗"
            formatted_key = key.replace('_', ' ').title()
            print(f"  {status} {formatted_key}: {value}")
    
    return analysis_results, plot_files

# Generate analysis and plots
analysis_results, plot_files = generate_analysis_and_plots(study, params)

## 5. Displaying Generated Plots

Let's display the key plots inline to see the results.

In [ ]:
# Display key plots inline
plots_dir = f"{params['output_dir']}/plots"

# List of key plots to display
key_plots = [
    'convergence_times_vs_weight.png',
    'costs_vs_weight.png',
    'performance_heatmap.png',
    'cost_convergence_tradeoff.png',
    'system_behaviour_radar.png'
]

print("Displaying key plots:")
for plot_name in key_plots:
    plot_path = f"{plots_dir}/{plot_name}"
    if os.path.exists(plot_path):
        print(f"\n{plot_name}:")
        img = plt.imread(plot_path)
        plt.figure(figsize=(12, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.show()
    else:
        print(f"\n{plot_name} not found at {plot_path}")

## 6. Output Structure

Here's what was generated and where to find it:


 output_path = Path(params['output_dir'])
    
All results are saved in: {output_path} \
    File structure: \
        {output_path}/" \
            ├── plots/") \
            │   ├── convergence_times_vs_weight.png") \
            │   ├── final_entropy_vs_weight.png") \
            │   ├── costs_vs_weight.png") \
            │   ├── performance_heatmap.png") \
            │   ├── system_behaviour_radar.png") \
            │   ├── comparative_behaviour_radar.png") \
            │   ├── cost_evolution_comparison.png") \
            │   ├── cost_convergence_tradeoff.png") \
            │   ├── stability_performance_tradeoff.png") \
            │   ├── weight_comparison_overview.png") \
            │   └── performance_distributions.png") \
            ├── weight_study_raw_data.csv") \
            ├── weight_study_analysis.json") \
            └── comprehensive_analysis_report.txt")

**File descriptions:** \
plots/: All generated visualisations \
weight_study_raw_data.csv: Raw experimental data \
weight_study_analysis.json: Statistical analysis results \
comprehensive_analysis_report.txt: Detailed analysis report

**Key plots explained:** \
convergence_times_vs_weight.png: How weight affects convergence speed \
costs_vs_weight.png: How weight affects final system cost \
performance_heatmap.png: Normalised performance across all metrics \
system_behaviour_radar.png: Multi-dimensional system behaviour patterns \
cost_convergence_tradeoff.png: Trade-off between cost and convergence speed



## 7. Tutorial Summary

This tutorial has successfully demonstrated the Weight Parameter Study experiment. Here's what we accomplished:

### Key Findings:
- **Convergence Speed**: Higher weights (0.7-0.9) achieve significantly faster convergence
- **System Performance**: Weight = 0.9 typically provides optimal cost performance
- **Learning Stability**: Moderate weights (0.3-0.5) show better stability
- **Optimal Range**: Recommended range 0.7-0.9 for most applications

### What You Can Do Next:
- Examine the generated plots in the plots/ directory
- Review the analysis report for detailed findings
- Modify parameters and run again to explore different scenarios
- Use the raw data for further custom analysis
- Compare results with other experiments to understand system dynamics

### Expected Results:
The experiment typically shows:
1. **Convergence Speed**: Higher weights (0.7-0.9) achieve significantly faster convergence
2. **System Performance**: Weight = 0.9 typically provides optimal cost performance
3. **Learning Stability**: Moderate weights (0.3-0.5) show better stability
4. **Optimal Range**: Recommended range 0.7-0.9 for most applications

All results have been saved to the specified output directory for further analysis.